In [ ]:
import pandas as pd
import numpy as np
import csv

import requests
from bs4 import BeautifulSoup
import re
import urllib.request

from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import ssl

In [ ]:
def base_ck(): #contest_korea사이트 크롤링
    title_bef =[]
    days_bef = []
    start_bef = []
    end_bef = []
    conditions = []
    inst_bef = []
    link_bef = []
    page_num = 1
    while(page_num<=5):
        url = 'https://www.contestkorea.com/sub/list.php?displayrow=12&Txt_sGn=1&Txt_key=all&Txt_word=&Txt_bcode=030210001&Txt_code1%5B0%5D=30&Txt_code1%5B1%5D=76&Txt_aarea=&Txt_area=&Txt_sortkey=a.int_sort&Txt_sortword=desc&Txt_chocode=&Txt_unicode=&page=' + str(page_num)
        req = urllib.request.urlopen(url)
        res = req.read()
        soup = BeautifulSoup(res,'html.parser')
        titles = soup.find_all("span", class_ = "txt")
        titles = list(titles)
        condition = soup.find_all("span", class_="condition")
        condition = list(condition)
        days = soup.select(".date > div" )
        days = list(days)
        
        inst = soup.select(".host > .icon_1" )
        inst = list(inst)

        for i in range(len(condition)):
            days[i] = (str(days[i]).strip('<div>').strip(">").strip("\t"))
            condition[i] = (str(condition[i]).strip('<span class="condition">').strip("</span>"))
            if condition[i] != '접수종료':
                conditions.append((condition[i]))
                titles[i] = (str(titles[i]).strip('<span class="txt">').strip("</'").strip(" "))
                days[i] = (str(days[i]).strip('<div>').strip('\n\t').strip('</li>'))
                inst[i] = (str(inst[i]).strip('<li class="icon_1"><strong>주최</strong> . ').strip('<div>'))
                if '공모전' not in titles[i]:
                    title_bef.append(titles[i])
                    start_day,end_day = days[i].split('~')
                    start_bef.append(start_day.replace(".", ". "))
                    end_bef.append(end_day.replace(".", ". ").strip('\t'))
                    inst_bef.append(inst[i])             
        page_num+=1
    return title_bef, start_bef, end_bef, inst_bef        

In [ ]:

def link_ck():
    conditions=[]
    link_bef = []
    page_num = 1
    while(page_num<=5):
        url = 'https://www.contestkorea.com/sub/list.php?displayrow=12&Txt_sGn=1&Txt_key=all&Txt_word=&Txt_bcode=030210001&Txt_code1%5B0%5D=30&Txt_code1%5B1%5D=76&Txt_aarea=&Txt_area=&Txt_sortkey=a.int_sort&Txt_sortword=desc&Txt_chocode=&Txt_unicode=&page=' + str(page_num)
        req = urllib.request.urlopen(url)
        res = req.read()
        soup = BeautifulSoup(res,'html.parser')
        titles = soup.find_all("span", class_ = "txt")
        titles = list(titles)
        condition = soup.find_all("span", class_="condition")
        condition = list(condition)
        for i in range(len(condition)):
            condition[i] = (str(condition[i]).strip('<span class="condition">').strip("</span>"))
            if condition[i] != '접수종료':
                conditions.append((condition[i]))
                titles[i] = (str(titles[i]).strip('<span class="txt">').strip("</'").strip(" "))
                if '공모전' not in titles[i]:                 #공모전이 붙어있는 것들은 제외해야 함.(공모전 크롤링 사이트가 따로 있으므로)
                    browser = Chrome('./chromedriver.exe')
                    delay=1
                    browser.implicitly_wait(delay)
                    browser.get(url)
                    browser.maximize_window()
                    link_adress = '//*[@id="frm"]/div/div[4]/ul/li['+ str(i+1) + ']/div[1]/a' #해당 게시물 링크의 path를 가지고 오는 코드
                    #사이트 변함 수정 필요
                    browser.find_elements_by_xpath(link_adress)[0].click()
                    sleep(0.5)
                    link_bef.append(browser.current_url)
                    
                    
        page_num+=1
    return link_bef

In [ ]:
def crawling_ck():
    title_bef, start_bef, end_bef, inst_bef = base_ck()
    link_bef = link_ck()
    
    tabl_data_bef = {'type':"1 경진대회", 'qualification': "대학(원)생",'title':  title_bef, 'notice': start_bef,  'deadline': end_bef, 'dday':0,'sponsor':inst_bef, 'title2': title_bef,
                     'link':link_bef}

    df_bef = pd.DataFrame(tabl_data_bef, columns=['type', 'qualification', 'title', 'notice', 'deadline', 'dday', 'sponsor', 'title2', 'link'])
    return df_bef

In [ ]:
//*[@id="frm"]/div/div[5]
//*[@id="frm"]
//*[@id="frm"]/div/div[5]/ul/li[1]/div[1]/a
/html/body/div[2]/div[3]/div[1]
//*[@id="wrap"]/div[3]/div[1]/div[1]
//*[@id="wrap"]/div[3]/div[1]/div[2]
//*[@id="wrap"]/div[3]/div[2]
//*[@id="frm"]/div/div[5]/ul/li[1]/div[1]
//*[@id="frm"]/div/div[5]/ul/li[1]/div[2]
//*[@id="frm"]/div/div[4]